* Structured Query language
* SQL has
    - Data definition language for defining relation schemas, deleting relation, modifying relation
    - Data manipulation language, query info from database. insert tuples, delete tuples, modifying tuples.
    - Integrity, We can provide integrity contraints.
    - Transaction control
        - It is sequence of SQL queries. When first SQL query executed transaction begin implicitly. Commit current transaction, meaning update performed by transaction become permanent in db. After transaction is committed new transaction automatically started.
        - Rollback cause current transaction to be rolled back, undoes all the updates performed by SQL statement in the transaction.
        - After transaction is committed we can not roll it back.
        - Transaction provides atomicity. Indivisible unit of transactions. All or none.
    - Authorization
* Always ask,
    - What tables do we need?
    - How to combine tables
    - What fields needed/ need to create
    - What filters need to be included
    * Ordering/limiting?

* SQL server is relational db system developed by microsoft
* Transact-SQL is microsoft's implementation of SQL.
* SELECT TOP(5) * FROM artists; # same as LIMIT
* SELECT TOP(5) PERCENT artist FROM artists;
* CHARINDEX('_', col_name) # find index of _

## Data definition
* We define schema of each relation
* Type of value associated with each attribute
* Integrity contraints.
* security and authorization for each relation.
* Physical storage structure on disk

### Types
* `char(n)` / `character(n)` : A fixed length character string.
    - If we provide shorter string, spaces will be appended.
    - Before comparing 2 char values, shorter string will be appended by spaces.
* `varchar(n)` / `character varying(n)` : variable length character with max length n.
* `int` / `integer`
* `small int`
* `numeric(p,d)` : fixed point number. it has p digits(with sign). d of the p digits are right side of decimal point.
* `real` / `double precision` 
* `float(n)` : floating  number with precision of n digits.
* `nvarchar`: stores unicode character, many SQL supports unicode in varchar only.
* `text` character string of any length
* `boolean` TRUE, FALSE, NULL
* `date`, `time`, `timestamp`

* general form
```
    CREATE TABLE r
        (A1 D1,
         A2 D2,
         ......,
         An Dn,
         <integrity constraints1>
         <integrity constraintsn>);
```

```
SELECT table_schema, table_name
FROM information_schema.tables;
```
* information_schema is meta-database that holds info about current database
    - It has multiple table which we can query
        - `tables`: Info about all tables in current db.
        - `columns` : Info about all columns in all tables in current database
* To look at columns of certain table

```
SELECT table_name, column_name, data_type
FROM information_schema.columns
WHERE table_name = 'department'
```
![](images/schema.jpg)

* 'public' schema holds information about user defined database and tables.
![](images/public_schema.jpg)

![](images/1.jpg)

### CREATE TABLE
```
CREATE TABLE department 
    (dept_name varchar(20) PRIMARY KEY,
     building varchar(15),
     budget numeric(12,2));
```
* In postgre we can checkout schema using `\d+ table_name
![](images/department.jpg)

```
CREATE TABLE course
    (course_id varchar(7) PRIMARY KEY,
     title varchar(50),
     dept_name varchar(20) REFERENCES department
     credits numeric(2,0));
```

* For each course tuple, the department name specified in tuple must exist in primary key attribute of department table.
![](images/course.jpg)

```
CREATE TABLE instructor
    (ID varchar(5) PRIMARY KEY,
     name varchar(20) NOT NULL,
     dept_name varchar(20) REFERENCES department (dept_name)
     salary numeric(8,2));
```
![](images/instructor.jpg)

* For some database (SQLite) `PRAGMA TABLE_INFO(table_name)` shows info of table and each columns

### Integrity constraints:
* Maintain data consistency. Guard against accidental damage to data.
* Most basic way to enforce integrity is thorugh data type of columns
* It can be added with CREATE TABLE and also with ALTER TABLE table_name ADD CONSTRAINT.
    - NOT NULL
    - UNIQUE
    - CHECK (predicate)
        - CHECK (budget > 0)
        - CHECK (semester IN ('Fall', 'Spring'))
* **primary key(A1, A2 ..., An)**:
    - Specify primary key attribute(s) for relation. It should be non null and unique.
* **foreign key(A1, A2, ... An) references s**:
    - Values for attributes(A1, A2, .. An) for any tuple in the relation must be corresponds to value of primary key attributes of some tuple in relation s.
    - Also known as referential integrity constraints
    - Ensure that value in one relation for given set of attributes also appears for a certain set of attributes in another relation.
    - Generally it points to PRIMARY KEY attribute of other tables.
    - It can also point to candidate key of other which is defined as UNIQUE constraint.
    - When violation occur transaction will be rolled back
    - Using deferred check we can check referential constraint at the end of transaction.
* **not null**: Null value not allowed for attribute.
* unique constraint:
    - Disallow duplicate in a column.
* Constraints give data structure
* Helps with consistency and so data quality.

#### Surrogate keys
    - System generated unique identifiers, internal identifiers
    - Like artificial primary key. They are not based on native column of your data.
    - We create new column which will act as primary key.
    
    ```
    ALTER TABLE table_name
    ADD COLUMN id serial PRIMARY KEY;
    ```
    
    ```
    ALTER TABLE table_name
    ADD COLUMN column_c varchar(256);
    
    UPDATE table_name
    SET column_c = CONCAT(columna, columnb);
    
    ALTER TABLE table_name
    ADD CONSTRAINT pk PRIMARY KEY  (column_c);
    ```

* DEFAULT is used to specify defuult value to insert into column when no explicit value is supplied.
* CHECK clause is used to designate conditions that must be met before data can be assigned to column.
* AUTOINCREMENT will increment attribute by 1 automatically, we do not need to specify it.

### INSERT INTO

```
INSERT INTO department (dept_name, building, budget) VALUES ('Biology', 'Watson', 90000);
INSERT INTO department (dept_name, building, budget) VALUES ('Computer', 'Taylor',100000);
INSERT INTO department (dept_name, building, budget) VALUES ('Electrical', 'Taylor', 85000)
INSERT INTO department (dept_name, building, budget) VALUES ('Fianace', 'Painter', 120000);
```

![](images/select_department.jpg)

* We can also INSERT rows from other table.

```
INSERT INTO table_name
SELECT DISTINCT firstnam,e lastname 
FROM other_table;
```
* Last 2 lines select all distinct values and INSERT statement append it to table_name
* Adding multiple rows at the same time
```
INSERT INTO department (dept_name, building, budget) 
VALUES 
    ('Fianace', 'Painter', 120000),
    ('Fianace', 'Painter', 120000);
    
# add roes from another table.
INSERT INTO top_companies
SELECT rank, title 
  FROM fortune500 
 WHERE rank BETWEEN 11 AND 20;
```

### DELETE 

```
DELETE FROM department
WHERE P;
```

```
DELETE FROM department;
```
* Delete all rows from table

### DROP
```
DROP TABLE department
```
* Remove entire table (including schema) from database.

### ALTER TABLE
```
ALTER TABLE table_name ADD COLUMN col_name varchar(20); #All row are assigned null for new attribute.
ALTER TABLE table_name DROP COLUMN col_name; # Many database does not support this operation 

ALTER TABLE table_name RENAME COLUMN old_name TO new_name;

ALTER TABLE table_name ALTER COLUMN name TYPE new_type;

# It will simply discard fractional part
ALTER TABLE table_name
ALTER COLUMN avg_grade
TYPE integer

# To round values

ALTER TABLE table_name
ALTER COLUMN avg_grade
TYPE integer
USING ROUND(avg_grade);

ALTER TABLE table_name
ALTER COLUMN col_name
TYPE varchar(x)
USING SUBSTRING(col_name FROM 1 FOR x) # reduce column data to first x character long

ALTER TABLE table_name
ALTER COLUMN col_name
SET NOT NULL;

ALTER TABLE table_name
ADD CONSTRAINT some_name UNIQUE(col_name);

ALTER TABLE table_name
ADD CONSTRAINT some_name PRIMARY KEY (col_name)

ALTER TABLE table_name
ADD CONSTRAINT some_name FOREIGN KEY (col_name) REFERENCES other_table (primary_col_name);

ALTER TABLE affiliations
DROP CONSTRAINT affiliations_organization_id_fkey;
```

### Comments
* single line comment
    - `-- comment`
    
* multi line comment
    - `/* I am comment */`

### UPDATE

```
UPDATE customer
SET phone = "+55 (12) 3921-4464"
WHERE customer_id = 1

UPDATE track
SET unit_price = (
                    SELECT AVG(unit_price)
                    FROM track
                 )

UPDATE track
SET unit_price = unit_price * 1.1

UPDATE wishlist_track
SET
    active = 1,
    purchased = 0;
```
* If you do not specify WHERE then all rows will be updated

### Large object type
* CLOB character data
* BLOB binary data

```
book_review CLOB(10KB)
image BLOB(10MB)
movie BLOB(2GB)
```
* Basically we retrieve locator of such object using SQL and with application we manipulate data at that locator.

### Our relations

![](images/instructor_1.jpg)
![](images/course_1.jpg)
![](images/prereq.jpg)
![](images/department_1.jpg)
![](images/section.jpg)
![](images/teaches.jpg)

### SELECT
* List attributes desired in result of query.
* If table name or columns name has spaces use `""` to enclose that.

```
SELECT name FROM instructor; # returns single attribute

SELECT dept_name FROM instructor;  # list department name for once every tuple, may be duplicated
```
* Above can also be written as,

```
SELECT ALL dept_name FROM instructor;
```

* To get it unique

```
SELECT DISTINCT dept_name FROM instructor;
```

* SELECT can contains arithmetic operation

```
SELECT ID, name, dept_name, salary*1.1 FROM instructor; # 10% hike in salary
```
* salary*1.1 is also known as derived/calculated column that is manipulation of existing columns in database. Operator +, *, -, / is useful to generate derived column, like we can sum two columns etc.

```
select ROUND(SALEPRICE) from PETSALE

select LENGTH(ANIMAL) from PETSALE

select UCASE(ANIMAL) from PETSALE

select * from PETSALE where LCASE(ANIMAL) = 'cat'
```

### LIMIT
* Limit the number of rows to be returned as a result of query
* postgre, SQLite
```
SELECT name
FROM instructor
LIMIT 10;
```
* Oracle uses `WHERE ROWNUM <= 10`
* DB@ uses `FETCH FIRST 10 ROWS ONLY`

### WHERE
* allows us to select rows which pass specific predicate.

```
SELECT name 
FROM instructor 
WHERE dept_name = 'Computer' and salary > 70000;
```

* We can use `and`, `or`, `not` as logical connectives.
* As comparison operator we can use <, <=, >, >=, =, !=
* Filtering data will dramatically speed up query so, get rid of unncessary data as much as  possible.
* To test your query run on smaller subset of data.
* LIMIT WORK after all other steps executed, to limit number of rows before the aggregation executed use sub query.

#### Retrieve name of all instructor along with dept_name and dept building name

```
SELECT name, instructor.dept_name, building 
FROM instructor, department
WHERE instructor.dept_name = department.dept_name;
```

### FROM
* Lists table which will be accessed during query.

```
SELECT A1, A2, An
FROM r1, r2
WHERE p1;
```

### Date and Time
*  DATE, TIME, and TIMESTAMP types
* DATE has 8 digits: YYYYMMDD
* TIME has six digits: HHMMSS
* TIMESTAMP has 20 digits: YYYYXXDDHHMMSSZZZZZZ where XX represents month and ZZZZZZ represents microseconds.
* Functions exist to extract the DAY, MONTH, DAYOFMONTH, DAYOFWEEK, DAYOFYEAR, WEEK, HOUR, MINUTE, SECOND.

```
select DAY(SALEDATE) from PETSALE where ANIMAL = 'Cat'

select COUNT(*) from PETSALE where MONTH(SALEDATE)='05'

select (SALEDATE + 3 DAYS) from PETSALE

select (CURRENT DATE - SALEDATE) from PETSALE
```

![](images/date_time_func.jpg)
![](images/timestring.jpg)

#### STRFTIME extract certain part of date
```
STRFTIME('%Y, birthdate) as Year
STRFTIME('%m, birthdate) as Month
STRFTIME('%d, birthdate) as day
SELECT DATE('now')
DATE(('now') - Birthdate) AS age

```

```
SELECT now(), now() + '5 minutes'::interval;
SELECT now(), now() + '100 days'::interval;
```

### Operational order of query
* First FROM then WHERE then SELECT
* FROM defines cartesian product of table mentioned.
![](images/from.jpg)
* In such cartesian product we combine tuple which are not related to each other, cartesian product of t1 and t2. Each row of t1 is combined with each row of t2.
* Such result is large and useless most of the times.
* Predicate from WHERE is used to restrict such combinations.
* For each tuple in result, output the attributes specified in SELECT.
* Actual SQL does not generate entire cartesian product, it will only generate product which is constrained by the WHERE.

### Natural join
* We usually write,
```
SELECT name, course_id
FROM instructor, teaches
WHERE instructor.ID == teaches.ID;
```

* Both table has the same attribute ID. Most of the time we want to combine our table on such same attribute.
* natural join is useful in such time.
* Cartesian product combine each tuple of instructor with each of teaches.
* Natural join considers only those pair of tuples with the same values on those attributes that appear in the schemas of both table. In instructor and teaches table, natural join will consider tuples where both the tuple from instructor and the tuple from teaches has same value on common attribute ID.

```
SELECT name, course_id
FROM instructor NATURAL JOIN teaches;
```

* In general

```
SELECT A1, A2, An
FROM t1 natural join t2 natural join tn
WHERE p;
```

#### List name of instructor along with titles of courses they teach

```
SELECT name, title
FROM instructor natural join teaches, course
WHERE teaches.course_id = course.course_id;
```

or 

```
SELECT name, title
FROM instructor natural join teaches natural join course;
```
* One problem is first natural join of instructor and teaches will happen, which yields relation,
![](images/natural_join.jpg)

* another natural join with course, will have common attributes dept_name  and course_id, natural join will require to have both attributes same in both intermediate and course relation. It will omit all the pair where instructor teaches a course in a department other than instructor's own department.

* To choose attribute on which we want to join we can use `JOIN... USING`

```
SELECT name, title
FROM (instructor natural join teaches) join course using (course_id); 
```


### Rename operation

```
SELECT name AS instructor_name, course_id
FROM instructor, teaches
WHERE instructor.ID = teaches.ID;
```

* Also we can use `AS` to rename long relation name

```
SELECT name AS instructor_name, course_id
FROM instructor AS t1, teaches AS t2
WHERE t1.ID = t2.ID;
```

* Useful when we want to compare relation with itself
- Find all the instructor name whose salary is more than at least one instructor from biology department.

```
SELECT DISTINCT T.name
FROM instructor AS T, instructor AS S
WHERE T.salary > S.salary and S.dept_name = 'Biology';
```

### LIKE
* Similar to WHERE =, but useful when we do not know what exactly we are looking for

### ILIKE
* Case insensitive matching. Bit slower than LIKE

### IN
* Similar to WHERE =, but more than 1 condition

```
SELECT *
FROM accounts
WHERE name IN ('Walmart', 'Google');
```
### NOT
* Used with IN and LIKE as NOT LIKE and NOT IN.

### AND 
### BETWEEN
* combines operation where all condition must be true
* It is inclusive.

### OR
* Combine operation where at least one condition must be true

### String operation
* 'Purvil', "Purvil's"
* Equality is case sensitive.
* Concatenate using `||`.
* Extracting substring
* finding length of string
* `upper(s)` `lower(s)`
* `trim(s)` Removing space at the end of string.
* pattern matching `like`
    - `%` matches any substring
    - `_` matches any character
    - `Intro%` matches string beginning with Intro.
    - `%Comp%` matches any string containing Comp as substring
    - `___` matches string of 3 character long
    - `___%` string at least 3 character long
   
   
```
SELECT dept_name
FROM department
WHERE building LIKE `%Watson%`;
```
* Escape character is used to treat special character as normal character.
    - `ab\%cd%` matches all string beginning with 'ab%cd'
* `NOT LIKE` is used to search mismatch.

### ORDER
```
SELECT name
FROM instructor
WHERE dept_name = 'Physics'
ORDER BY name;
```

```
SELECT name
FROM instructor
WHERE dept_name = 'Physics'
ORDER BY name DESC;
```

```
SELECT name
FROM instructor
WHERE dept_name = 'Physics'
ORDER BY salary DESC, name ASC;
```

* We can also use column number , `ORDER BY 2` meaning order by second colummn.

### BETWEEN

```
SELECT name
FROM instructor
WHERE salary BETWEEN 90000 AND 100000;
```

* Similarly use `NOT BETWEEN`

* Find instructor name and course they teach in the biology department who have tought some courses
```
SELECT name, course_id
FROM instructor, teaches
WHERE instructor.ID = teaches.ID AND dept_name = 'Biology';
```

```
SELECT name, course_id
FROM instructor, teaches
WHERE (instructor.ID, dept_name) = (teaches.ID, 'Biology');
```
* Checks instructor.ID = teaches.ID and dept_name = 'Biology'

### Set Operation
* Set of all course taught in Fall 2009 semester
```
SELECT course_id
FROM section
WHERE semester = 'Fall' and year = 2009;
```

* Set of all course taught in Spring 2010 semester
```
SELECT course_id
FROM section
WHERE semester = 'Spring' and year = 2010;
```

#### Union
* All courses in fall 2009 and spring 2010 or both.
* Automatically eliminates duplicate
* Parenthesis is optional.
* Same nunmber of columns, same type of columns, Columns name can be different.
```
(SELECT course_id
FROM section
WHERE semester = 'Fall' and year = 2009)
UNION
(SELECT course_id
FROM section
WHERE semester = 'Spring' and year = 2010);
```

#### UNION ALL
* To retain duplicate use UNION ALL
```
(SELECT course_id
FROM section
WHERE semester = 'Fall' and year = 2009)
UNION ALL
(SELECT course_id
FROM section
WHERE semester = 'Spring' and year = 2010);
```

#### INTERSECT / INTERSECT ALL
* Courses that taught in both. Eliminates duplicate. Use `INTERSECT ALL` to retain duplicate.

```
(SELECT course_id
FROM section
WHERE semester = 'Fall' and year = 2009)
INTERSECT
(SELECT course_id
FROM section
WHERE semester = 'Spring' and year = 2010);
```

#### EXCEPT
* All courses in first relation but not from second
* Eliminates duplicate before doing set difference. If 4 same course in 1st and 2 same course in second. None of that will be in output. To avoid it use `EXCEPT ALL`
```
(SELECT course_id
FROM section
WHERE semester = 'Fall' and year = 2009)
EXCEPT
(SELECT course_id
FROM section
WHERE semester = 'Spring' and year = 2010);
```

### Null values
* Arithmetic operation with Null is Null.
* Comparison operator with Null, yields Unknown.
* True and Unknown is Unknown. False and Unknown is False.
* Unknown and Unknown is Unknown.
* True OR Unknown is True, False OR Unknown is Unknown. Unknown OR Unknown is Unknown.
* not Unknown is Unknown.
* If WHERE case predicate evaluates to False or Unknown, that tuple will be ignored.
* To test a null value,
```
SELECT name
FROM instructor
WHERE salary IS NULL;
```
* Same we can use `IS NOT NULL`.
* Some SQL allows `IS UNKNOWN` and `IS NOT UNKNOWN`.

### Aggregate Function
* Takes collection of values as input and return single value.

- `AVG`, `MIN`, `MAX`, `SUM`, `COUNT`
* AVG and SUM needs collection of numbers, others can work with strings also.
* `TOTAL` returns floating point sum
* Find average salary of instructor in computer science department
```
SELECT AVG(salary) AS avg_salary 
FROM instructor
WHERE dept_name = 'Computer';
```
* Of course we have to consider duplicated when we compute avg.
* But for some case we have to get rid of duplicates. Use `DISTINCT` for that
* Find total instructor who teach a course in the spring 2010 semester
```
SELECT COUNT(DISTINCT ID)
FROM teaches
WHERE semester = 'Spring' AND year = 2010;
```

* To count number of rows
```
SELECT COUNT(*)
FROM course;
```
* We can NOT use DISTINCT with *  or with MAX, MIN.
* AVG ignores NULL, to consider NULL in denomintor use SUM(col_name) / COUNT(*)
* DISTINCT always apply on all selected column.
* DISTINCT slow query execution.

### Variance/ Standard deviation

```
SELECT var_pop(col_name)
FROM table_name

SELECT var_samp(col_name) # We can also use variance(col_name)
FROM table_name


SELECT stddev_pop(col_name)
FROM table_name

# For sample standard deviation, stddev_samp or stddev
```

### Truncate
* Reduces precision
* Replace right most digits with 0. We will never get largest abs value than  original number

```
SELECT trunc(42.1256, 2); # truncate 2 digit after decimal

# return 42.12

SELECT trunc(12345,-3); # 12000 # replace 3 digit left to decimal to 0
```
* One of the technique to group digits and count it

### generate_series

```
SELECT generate_series(start, end, step); # inclusive
```
* It is useful to create bins.

```
-- Create bins
WITH bins AS (
      SELECT generate_series(30,60,5) AS lower,
             generate_series(35,65,5) AS upper), 
     -- Subset data to tag of interest
     ebs AS (
      SELECT unanswered_count
        FROM stackoverflow
       WHERE tag='amazon-ebs')
-- Count values in each bin
SELECT lower, upper, count(unanswered_count) 
  -- left join keeps all bins
  FROM bins
       LEFT JOIN ebs
              ON unanswered_count >= lower
             AND unanswered_count < upper
 -- Group by bin bounds to create the groups
 GROUP BY lower, upper
 ORDER BY lower;
```

```
SELECT generate_series('2018-01-01',
                       '2018-01-02',
                       '5 hours'::interval);
```
### corr
* take two col as an argument and return correlation between them
* Correlation coefficient range from -1 to 1. more negative relation closer to -1. more positive closer to 1.
```
SELECT corr(col_A, col_B)
FROM table_name;
```
* Rows with null values are excluded.

### median

```
SELECT percentile_disc(percentile) WITHIN GROUP (ORDER BY col_name)
FROM table_name;
```
* percentile parameter is between 0 and 1.
* `percentile_disc` return value from column.
* `percentile_cont` interpolates between values.

```
val 
-----
   1
   3
   4
   5
   
SELECT percentile_disc(.5) WITHIN GROUP (ORDER BY val),
       percentile_cont(.5) WITHIN GROUP (ORDER BY val)
  FROM nums;
 
 percentile_disc | percentile_cont 
-----------------+-----------------
               3 |             3.5
```

#### Finding median
```
SELECT *
FROM (SELECT total_amt_usd
      FROM orders
      ORDER BY total_amt_usd
      LIMIT 3457) AS Table1
ORDER BY total_amt_usd DESC
LIMIT 2;
```

### Type CAST
* To cast entire column

```
SELECT CAST(col_name AS integer)
FROM table;
```

```
SELECT numeric_col * CAST(text_col AS integer) AS multiply_col
FROM table;
```

* Another way is `SELECT value::new_type;`

### Grouping
* Some time we want to aggregate group of tuples. Use `GROUP BY`.
* Tuple with same value on all attributes in GROUP BY clause will places in one group.
* Find avg salary in each department
```
SELECT avg(salary) as avg_salary
FROM instructor
GROUP BY dept_name;
```
* Find number of instructor in each department who teach a course in the spring 2010 semester.
```
SELECT dept_name, COUNT (DISTINCT ID) AS inst_count
FROM instructor NATURAL JOIN teaches
WHERE semester = 'Spring' and year = 2010
GROUP BY dept_name;
```
* MAKE SURE ONLY ATTRIBUTE IN SELECT STATEMENT WHICH ARE NOT AGGREGATED ARE FROM GROUP BY STATEMENT.  ATTRIBUTE WHICH ARE NOT IN GROUP BY CAN PRESENT WITH AGGREGATION ONLY, OTHERWISE ERROR WILL OCCUR.
* For some SQL implementation we can select column which is not in group by it will use last value of that column.

### HAVING
* When we want to apply condition on group NOT on tuple, HAVING is useful.
* Useful to apply filter based on aggregate function. We can NOT use aggregate function with WHERE.
* Department whose avg salary is $42000. SQL apply predicate of HAVING after the GROUP BY statement is executed.

```
SELECT dept_name, avg(salary) AS average_salary
FROM instructor
GROUP BY dept_name
HAVING avg(salary) > 42000;
```
* Attribute in HAVING that are not aggregated must be present in GROUP BY
* FROM clause is evaluated first to get relation.
* If the WHERE is present, predicate of WHERE is applied on relation of FROM.
* Tuple which satisfy WHERE are placed into groups using GROUP BY. If no GROUP BY then entire list of tuple is counted as single group.
* The HAVING clause  is applied to each group, if group do not satisfy HAVING, that will be discarded.
* SELECT clause uses the remaining group to generate tuples of the result of the query, applying aggregation to gent single result tuple from each group.

* For each course section offered in 2009, find average total credit of all students enrolled in the section, if the section has at least 2 students.

```
SELECT course_id, semester, sec_id, year, avg(tot_cred)
FROM student NATURAL JOIN takes
WHILE year = 2009
GROUP BY course_id, semester, sec_id, year
HAVING COUNT(ID) >= 2
```

* All aggregate function except COUNT ignore the null value.
* Count of empty collection is 0, all other aggregate function return NULL when we supply empty collection.
* Boolean data type can takes TRUE, FALSE and UNKNOWN
* Aggregate function SOME and EVERY can work with boolean values.

### Nested sub-queries
* SELECT_FROM_WHERE expression that is nested within other query.
* Hard coded values are good candidate to replace with sub queries.
* SELECT rows based on average value of some column, We can only use aggregation in SELECT or HAVING clause, we can not use in WHERE clause as in SELECT * FROM table_name WHERE col_name > AVG(col2_name)
* Select specific records and use that as filtering criteria of next query.
* We must have to use subquery 
```
SELECT * FROM table_name WHERE col_name >
(SELECT AVG(col2_name) FROM table_name)
```
* Subquery acts as dynamic filter criteria. 

#### Set membership
* Find all the courses taught in the both fall 2009 and spring 2010.
* Test membership of tuples of one relation to another relation.
* We use IN and NOT IN to check for membership
```
SELECT DISTINCT course_id
FROM section
WHERE semester = 'Fall' AND year = 2009 AND course_id IN
(SELECT * 
FROM section
WHERE semester = 'Spring' AND year = 2010)
```

* Find all courses taught in Fall 2009 semester but not in spring 2010.
```
SELECT DISTINCT course_id
FROM section
WHERE semester = 'Fall' and year = 2009 AND course_id NOT IN
(SELECT course_id
FROM section
WHERE semester = 'spring' AND year = 2010)
```

* Find the total number of distinct students who have taken course section taught by instructor with ID 10101.

```
SELECT COUNT(DISTINCT ID)
FROM takes
WHERE (course_id, semester, year) IN
(SELECT course_id, semester, year
FROM teaches
WHERE ID = 10101);
```

* Get region of each customer who has had an order with freight over 100.

```
SELECT CustomerID, CompanyName, Region FROM Customers WHERE CustomerID IN (SELECT customerID FROM orders WHERE Freight > 100);
```
#### Set Comparison
* Find name of all instructor whose salary is greater than at least one instructor in biology department.

```
SELECT T.name
FROM instructor AS T, instructor AS S
WHERE T.salary > S.salary AND S.dept_name = 'Biology'
```

```
SELECT name
FROM instructor
WHERE salary
> SOME (SELECT salary
FROM instructor
WHERE sept_name = 'Biology')
```

* We can use SOME and ALL with comparison operators.
* SOME checks for at least one, ALL checks for all.
* `=SOME` is same as IN and <>SOME is same as NOT IN.
* Names of all instructor that have salary value greater than each instructor in biology dept

```
SELECT name 
FROM instructor
WHERE salary > ALL
(SELECT salary
FROM instructor
WHERE dept_name = 'Biology');
```

* Find all department that have highest average salary

```

SELECT dept_name
FROM instructor
GROUP BY dept_name
HAVING AVG(salary) >=ALL
(SELECT AVG(salary)
FROM instructor
GROUP BY dept_name);
```

#### Test for empty relation
* Check whether sub query has any tuple in its result?
* EXISTS construct returns True if argument subquery is non empty.

* Find all courses taught in both the fall 2009 semester and spring 2010.

```
SELECT course_id
FROM section AS S
WHERE semester = 'Fall' AND year = 2009 AND EXISTS
(SELECT course_id
FROM section AS T
WHERE semester = 'spring' AND year = 2010 AND S.course_id = T.course_id)
```
* Subquery uses the relation name from outer , such sub query is called correlated subquery.
* NON EXISTS construct is used to check non existence
* Find all students who have taken all courses offered in biology department
* To check whether relation A contains entire relation B, use `B EXCEPT A`
```
SELECT S.ID, S.name
FROM STUDENT as S
WHERE NOT EXISTS (
(SELECT course_id
FROM course
WHERE dept_name = 'Biology')
EXCEPT
(SELECT T.course_id
FROM takes as T
WHERE S.ID = T.ID))
```
![](images/corelated.jpg)


#### Test for absent of duplicate tuple
* UNIQUE construct returns the value True if the argument sub query contains no duplicate value.
* Find all courses that were offered at most once in 2009.


### Subqueries in From clause
- The key concept applied here is that any select-from-where expression returns a relation as a result and, therefore, can be inserted into another select-from-where anywhere that a relation can appear.
- Find avg instructors salaries of those departments where the avg slary is more than 42000

```
SELECT dept_name, avg_slary
FROM (SELECT dept_name, AVG(salary) as avg_Salary
FROM instructor
GROUP BY dept_name)
WHERE avg_salary > 42000;
```
* Even we can name the inner table 

```
SELECT dept_name, avg_slary
FROM (SELECT dept_name, AVG(salary)
FROM instructor
GROUP BY dept_name
AS dept_avg(dept_name, avg_salary))
WHERE avg_salary > 42000;
```
* Oracle does not support renaming of result relation.


### Subqueries in With clause
* Defines temporary relation whose definition is available only to the query in which the with clause occurs.

```
with max_budget(value) AS
    (SELECT MAX(budget) FROM departmrent)
SELECT budget
FROM department, max_budget
WHERE department.budget = max_budget.value;
```

* Find all department where total_salary is greater than avg of total salary at all departments.

```
with dept_total(dept_name, value) AS 
    (SELECT dept_name, sum(salary) FROM instructor GROUP BY dept_name),
dept_total_avg(value) AS
    (SELECT AVG(value) FROM dept_total)
SELECT dept_name
FROM dept_total, dept_total_avg
WHERE dept_total.value >= dept_total_avg.value;
```

### Scalar subqueries
* subquery can occur whenever expression returning a value is permitted, provided the subquery returns only one tuple containing a single attribute.
* List all department along with number of instructor in each department

```
SELECT dept_name, (SELECT COUNT(*) FROM instructor WHERE department.dept_name = instructor.dept_name) AS num_isntructor
FROM department;
```

### Join

![](images/student.jpg)
![](images/takes.jpg)


#### Inner Join

```
SELECT *
FROM left_table
INNER JOIN right_table
ON left_table.id = right_table.id;
```


```
SELECT *
FROM left_table
INNER JOIN right_table
USING(id); # when column name is same in both table
```

```
SELECT * 
FROM student INNER JOIN takes ON student.ID = rakes.ID;
```
* Rows from students matches a tuple from takes if their ID value are equal. Same as `student NATURAL JOIN takes`
* Above query same as

```
SELECT *
FROM student, takes
WHERE student.ID = takes.ID
```
![](images/result.jpg)
* Joining two table without an ON clause gives all possible combinations of rows. Like cartesian product.

#### Outer Join
* Left outer join : preserve tuples only in the relation named at left of LEFT OUTER JOIN
    - attribute of left relation are filled in with thr value from right relation Remaining value of left relation will filled with NULL
    
    ```
    SELECT * FROM student NATURAL LEFT OUTER JOIN takes;
    ```
    
    ```
    SELECT * FROM student LEFT OUTER JOIN takes ON student.ID = takes.ID;
    ```
    ![](images/left_outer_join.jpg)
    
    - Find all student who has not taken any course
    
    ```
    SELECT ID FROM student NATURAL LEFT OUTER JOIN  takes
    WHERE course_id is NULL
    ```
* Right outer join : preserve tuples only in the relation named at right of LEFT OUTER JOIN
    - Tuples from right side relation that do not match any tuple from left relation are padded with nulls.
    
    ```
    SELECT * FROM takes NATURAL RIGHT JOIN student;
    ```
   ![](images/right_outer_join.jpg)
    
* FULL OUTER JOIN : preserve tuples in both relations.
     - Display list of students in the computer department along with the course section if any that they have taken in spring 2009
     
     ```
     SELECT * FROM (SELECT * FROM student where dept_name = 'computer') NATURAL FULL OUTER JOIN (SELECT * FROM takes WHERE semester = 'Spring' and year = 2009;)
     ```
* Full outer join can be used to quickly check is there any unmatched row between tables. Account and sales person, if not unmatch then each account has sales person 
* If you wanted to return unmatched rows only, which is useful for some cases of data assessment, you can isolate them by adding the following line to the end of the query:

```
WHERE Table_A.column_name IS NULL OR Table_B.column_name IS NULL
```
* List all students displaying their ID and name dept_name and tot_cred along with course that they have taken.

```
SELECT *
```


* To join more than 2 tables

```
SELECT [column_names] FROM [table_name_one]
[join_type] JOIN [table_name_two] ON [join_constraint]
[join_type] JOIN [table_name_three] ON [join_constraint];
```
* The SQL engine interprets joins in order, so the first join will be executed, and then the second join will be executed against the result of the first join
* We can also join a table using <=, >=, != instead of =, but it is very rare. ALso we can chain ON condition using AND OR.
```
SELECT accounts.name as account_name,
       accounts.primary_poc as poc_name,
       sales_reps.name as sales_rep_name
  FROM accounts
  LEFT JOIN sales_reps
    ON accounts.sales_rep_id = sales_reps.id
   AND accounts.primary_poc < sales_reps.name
```
* During join try to reduce rows to gain performance.Instead of after aggregation do pre aggregation and then join tables. Sub queries may be useful here.

#### Self join
* Useful when 2 events occur one after another
```
SELECT o1.id AS o1_id,
       o1.account_id AS o1_account_id,
       o1.occurred_at AS o1_occurred_at,
       o2.id AS o2_id,
       o2.account_id AS o2_account_id,
       o2.occurred_at AS o2_occurred_at
  FROM orders o1
 LEFT JOIN orders o2
   ON o1.account_id = o2.account_id
  AND o2.occurred_at > o1.occurred_at
  AND o2.occurred_at <= o1.occurred_at + INTERVAL '28 days'
ORDER BY o1.account_id, o1.occurred_at
```

```
SELECT
    e1.employee_id,
    e2.employee_id supervisor_id
FROM employee e1
INNER JOIN employee e2 on e1.reports_to = e2.employee_id
LIMIT 4;
```

### Cross join
![](images/cross_join.jpg)

```
SELECT *
FROM table1 
CROSS JOIN table2


SELECT *
FROM table1, table2

SELECT *
FROM table1, table2
WHERE table1.col1 = table2.col2
```

### Date
* When we have granularity up to the second level, grouping does not help much.
* `DATE_TRUNC('day', date_col)`
* Other available optionbs are second, month, year
* `DATE_PART` will return specific part of date.
    - DATE_PART('second', date_col)
    - We can also use day, month, year, dow (returns day of week returns from 0 to 6, 0 is sunday and 6 is saturday).
* `DATEADD` add or subtract datetime value. Always return date
    - `SELECT DATEADD(DD, 30, '2020-06-21')` Add 30 days to given date
* `DATEDIFF` Obtain difference between two datetime values. Always returns a number.
    - `SELECT DATEDIFF(DD, '2020-05-22', '2020-06-21') AS Difference1`
* `TO_DATE`

```
TO_DATE(month, col_month)
```
* It will convert January to 1 and so on.

### CASE 
* Derived column is take data from existing column and make new column.
* CASE is followed by WHEN and THEN statemetnt. Finish with word END.
* `CASE WHEN channel = 'facebook' OR channel = 'direct' THEN 'yes' ELSE 'no' END AS is_facebook`
* We can have many WHEN THEN statement
* CASE statement always in SELECT clause.
* Write a query to display for each order, the account ID, total amount of the order, and the level of the order - ‘Large’ or ’Small’ - depending on if the order is 3000 or more, or smaller than 3000.

```
SELECT account_id, 
       total_amt_usd, 
       CASE WHEN total_amt_usd > 3000 THEN 'Large' 
            ELSE 'Small' END AS level
FROM orders
```

* To save new column as new table use INTO


```
SELECT account_id, 
       total_amt_usd, 
       CASE WHEN total_amt_usd > 3000 THEN 'Large' 
            ELSE 'Small' END AS level
INTO new_table
FROM orders
```
* It will create temporary table named new_table, which will persist for database session.
* Another way is,
```
CREATE TEMP TABLE new_tablename AS
SELECT col1, col2
FROM table_name
```
* Using CASE filter the records.

```
SELECT *
FROM table
WHERE 
    CASE WHEN a > 5 THEN 'Keep'
         WHEN a <= 5 THEN 'Exclude' END = 'Keep';
```
* We can use aggregation with CASE too

```
SELECT
    season,
    SUM(CASE WHEN hometeam_id = 8634 
        THEN home_goal END) AS barca_homegoals,
    SUM(CASE WHEN awayteam_id = 8634 
        THEN away_goal END) AS barca_awaygoals
FROM match
GROUP BY season;


SELECT
    season,
    COUNT(CASE WHEN hometeam_id = 8634 AND home_goal > away_goal 
          THEN id END) AS home_wins,
    COUNT(CASE WHEN awayteam_id = 8634 AND away_goal > home_goal 
          THEN id END) AS away_wins
FROM match
GROUP BY season;


SELECT
    season,
    AVG(CASE WHEN hometeam_id = 8634 
        THEN home_goal END) AS barca_home_goals,
    AVG(CASE WHEN awayteam_id = 8634 
        THEN away_goal END) AS barca_away_goals
FROM match
GROUP BY season;


AVG(CASE WHEN condition_is_met THEN 1
         WHEN condition_is_not_met THEN 0 END) # Calculate percentage 
```

### With
*  With clauses allow you to define one or more named subqueries before the start of the main query. The main query then refers to the subquery by it's alias name, just as if it's a table in the database.
* It is called common table expression or CTE. Sub query is hard to read. Using CTE we can make sub query logic on its own and outer expression separated.

```
WITH [alias_name] AS ([subquery])

SELECT * FROM alias_name

WITH
    usa AS
        (
        SELECT * FROM customer
        WHERE country = "USA"
        ),
    last_name_g AS
        (
         SELECT * FROM usa
         WHERE last_name LIKE "G%"
        ),
    state_ca AS
        (
        SELECT * FROM last_name_g
        WHERE state = "CA"
        )

SELECT
    first_name,
    last_name,
    country,
    state
FROM state_ca
```

### CREATE VIEW
* permanently define sub query 
* We use it to encapsulate queries
* Can add or remove columns without changing schema
* View will be removed after database session ended.
```
CREATE VIEW database.view_name AS
SELECT * FROM database.table;

CREATE VIEW chinook.customer_2 AS
    SELECT
        customer_id,
        first_name || last_name name,
        phone,
        email,
        support_rep_id
    FROM chinook.customer;

DROP VIEW chinook.customer_2;
```
* Once a view is created it acts exactly like a table.
* Useful with complex multilevel query.
* When we do not have rights to create table.
* 

### `||`
```
SELECT ("this" || "is" || "my" || "string");
```
* It will return thisismystring

```
SELECT
    album_id,
    artist_id,
    "album id is" || album_id col_1,
    "artist id is" || artist_id col2,
    album_id || artist_id col3
FROM album LIMIT 3;
```

### CONCAT
* combine column together across the rows

```
CONCAT(col1, ' ', col2)
```

#### REPLACE

```
SELECT REPLACE(col, '.', '') AS col_name
```

#### LEFT
* To extract first `n` characters
```
SELECT LEFT(col, n) as new_col
```

#### RIGHT

#### SUBSTRING

```
SUBSTRING(fieldName from S for N)
```
* Returns the N characters starting from position S
* Both S and N is optional

#### SUBSTR(string, startPos, numberOfChar)

#### UPPER 

#### LOWER

#### INITCAP


#### TRIM
* Remove extra spaces
* Threre are RTRIM and LTRIM.
* To remove other characters to trim we can specify
    - `SELECT TRIM('WoW!', '!')`

#### LENGTH
* Returns length of the string.

#### POSITION
* Supply substring to search and returns starting position of substring from the left.
* Index of first character is 1 in SQL.
```
POSITION(',' IN col_name)
```

### STRPOS
* Same as above

```
STRPOS(city_state, ',')
```

### SPLIT_PART

```
SELECT split_part('a,bc,d', ',', 2);
returns bc
```

### Coalesce
* Return first non null value.

```
column_1 | column_2 
----------+----------
          |       10
          |         
       22 |         
        3 |        4
        
SELECT coalesce(column_1, column_2)
  FROM prices;

coalesce 
----------
       10

       22
        3
```
* Very useful to replace null value to something else during computation.


### ISNULL
* Replace null value to specific value

```
SELECT ISNULL(col, 'Unknown') AS newCol
```

### ABS(number)
* Absolute

### SQRT

### SQUARE

### LOG() 
* Return natural log
* `LOG(col, 10)` Base 10 logarithm

### Declare variable

```
DECLARE @var_name INT
SET @var_name = 5
DECLARE @var_name VARCHAR(100)
```
* To use variable

```
SELECT * FROM table
WHERE col_name = @var_name
```

```
-- Declare ctr as an integer
DECLARE @ctr INT
-- Assign 1 to ctr
SET @ctr = 1
-- Specify the condition of the WHILE loop
WHILE @ctr < 10
    -- Begin the code to execute inside WHILE loop
    BEGIN
       -- Keep incrementing the value of @ctr
       SET @ctr = @ctr + 1
       -- Check if ctr is equal to 4
       IF @ctr = 4
           -- When ctr is equal to 4, the loop will break
           BREAK
       -- End WHILE loop
    END
```

### psql Command line tool for postgresql
* To exit use `\q`
* `\l` : list all available databases.
* `\dt` : list all tables in the current database.
* `\du` : list users
* `\d+ tablename` : Schema of table
* `\dp tablename` privileges granted to users for specific table.
* To connect with specific database use, `psql -d dbname`
* To create user

```
CREATE ROLE username WITH LOGIN PASSWORD 'pwd'; 

# Providing ability to create database
CREATE ROLE username WITH CREATEDB LOGIN PASSWORD 'pwd'; 

CREATEROLE allow user to create other users
SUPERUSER makes user a superuser.
```

* Allow user to perform select query of table

```
GRANT SELECT, INSERT, UPDATE, DELETE ON tablename TO username;
GRANT ALL PRIVILEGES ON tablename TO userName;
```

* Remove permission

```
REVOKE SELECT, INSERT, UPDATE, DELETE ON tablename FROM username;
REVOKE ALL PRIVILEGES ON tablename FROM userName;
```

* To login with user name 

```
psql -U username
# After it enter password when prompted.
```

#### sqlite command line 
* To open database `sqlite3 dbname.db`
* By defualt column name will not be shown on SELECT query, to turn it on `.headers on`
* To print table line up `.mode column`
* `.help` Display help text showing all dot commands and their function
* `.tables` List of all tables and views in the current database
* `.shell [command]` Run a command like ls or clear in the system shell
* `.quit` Quit SQLite shell 
* `.shema table_name` to view schema for a table

### Case study
* Write a SQL statement that computes the proportion (as a float value) of rows that contain above average values for the ShareWomen.

```
SELECT COUNT(*) / CAST((SELECT COUNT(*) FROM recent_grads) AS FLOAT) AS proportion_abv_avg
FROM recent_grads
WHERE ShareWomen > (SELECT AVG(ShareWomen)
FROM recent_grads)
```
* Write a query that returns the Major and Major_category columns for the rows where:Major_category is one of the 5 highest group level sums for the Total column

```
SELECT Major, Major_category
FROM recent_grads
WHERE Major_category IN (SELECT Major_category
                        FROM recent_grads
                        GROUP BY Major_category
                        ORDER BY SUM(TOTAL) DESC
                        LIMIT 5)
```

### Ranking

```
SELECT ID, rank() OVER (ORDER BY (GPA) DESC)  AS s_rank
FROM table_name;
```
* To order entire relation we will have to use one more ORDER BY


```
SELECT ID, rank() OVER (ORDER BY (GPA) DESC)  AS s_rank
FROM table_name
ORDER BY s_rank;
```
* For same value it give same rank and skip next rank by repeated values. Or continue with next rank (Dense_rank).

```
SELECT ID, (1 + (SELECT COUNT(*) FROM table_name t WHERE t.GPA > a.GPA)) AS s_rank
FROM table_name a
ORDER BY s_rank;
```
* We can also rank after partitioning the data, Like rank by department instead full university.

```
SELECT ID, dept_name, rank() OVER(PARTITION BY dept_name ORDER BY GPA DESC) as dept_rank
FROM table_name
ORDER BY dept_name, dept_rank
```
* Group by apply first and ranking executed on grouped tuples.

#### percent_rank
* Fives rank of tuple as fraction. n total tuple, rank of tuple is r. Then page rank is (r-1) / (n-1)

#### cume_dist
* Cumulative distribution, if tuple is defined as p/n where p is number of tuples in the partition with 

### Windowing
* Compare one row to other row without join, useful for running total, find whether one row is greater than previous row.
* Window function perform calculation across a set of table rows that somehow related to current row.

### Running total

```
SELECT col_name, SUM(col_name) OVER (ORDER BY col2_name) AS running_total
FROM table_name
```
* Take the sum of col_name of all rows from beginning up to the current row in order by col2_name.

* I want to find running total for each month, meaning running total will reset at beginning of each month. We have to use partition by.

```
SELECT col_name, DATE_TRUNC('month', occurred_at) AS month,
        SUM(col_name) OVER (PARTITION BY DATE_TRUNC('month', occurred_at) ORDER BY occurred_at) AS running_total
FROM table_name
```

* `OVER` has 3 component 
    - Partitioning `PARTITION BY`. With GROUP BY only 1 row returns each group. PARTITION is just one way to divide the data to perform calculations.
    ![](images/partition.png)
    - ordering `ORDER BY`
    - framing
* We can not use window function in GROUP BY clause.

### ROW_NUMBER

```
SELECT id, account_id, occurred_at, ROW_NUMBER() OVER (ORDER BY occurred_at) AS row_num
FROM table_name

SELECT id, account_id, occurred_at, ROW_NUMBER() OVER (PARTITION BY account_id ORDER BY occurred_at) AS row_num
FROM table_name
```


### Rank

```
SELECT id, account_id, occurred_at, RANK() OVER (PARTITION BY account_id ORDER BY occurred_at) AS row_num
FROM table_name
```
* If two occurred_At is similar both will have same rank and next occurred_at will be skipped by one rank.

* To not skip values use `DENSE_RANK()`

```
SELECT id,
       account_id,
       standard_qty,
       DATE_TRUNC('month', occurred_at) AS month,
       DENSE_RANK() OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS dense_rank,
       SUM(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS sum_std_qty,
       COUNT(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS count_std_qty,
       AVG(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS avg_std_qty,
       MIN(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS min_std_qty,
       MAX(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS max_std_qty
FROM orders
```

![](images/answer1.jpg)

```
SELECT id,
       account_id,
       standard_qty,
       DATE_TRUNC('month', occurred_at) AS month,
       DENSE_RANK() OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS dense_rank,
       SUM(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS sum_std_qty,
       COUNT(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS count_std_qty,
       AVG(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS avg_std_qty,
       MIN(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS min_std_qty,
       MAX(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS max_std_qty
FROM orders
```
![](images/answer2.jpg)


* Removing ORDER by leaves an unordered partition.Each column's value will be just aggregation of all standard_qty values in respective account_id.
* leaving the ORDER BY out is equivalent to "ordering" in a way that all rows in the partition are "equal" to each other.

```
SELECT id,
       account_id,
       standard_qty,
       DATE_TRUNC('month', occurred_at) AS month,
       DENSE_RANK() OVER main_window AS dense_rank,
       SUM(standard_qty) OVER main_window AS sum_std_qty,
       COUNT(standard_qty) OVER main_window AS count_std_qty,
       AVG(standard_qty) OVER main_window AS avg_std_qty,
       MIN(standard_qty) OVER main_window AS min_std_qty,
       MAX(standard_qty) OVER main_window AS max_std_qty
FROM orders
WINDOW main_window AS (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at))
```

### Compare rows to other rows
* `LAG` returns value from previous row to the current row in the table.

```
SELECT account_id,
        standard_sum,
        LAG(standard_sum) OVER (ORDER BY standard_sum) as lag,
        LEAD(standard_sum) OVER (ORDER BY standard_sum) as lead
FROM 
```
![](images/lag_lead.jpg)

### Percentile
* `NTILE(# of buckets)` ORDER BY determine which column to use to determine quartiles.
* When there are fewer row than number of buckets, use fewer bands.

```
SELECT id, account_id, occurred_At, standard_qty,
        NTILE(4) OVER (ORDER BY standard_qty) as quartile
        NTILE(5) OVER  (ORDER BY standard_qty) as quintile,
        NTILE(100) OVER (ORDER BY standard_qty) AS percentile
FROM table_name
ORDER BY standard_qty DESC
```

![](images/percentile.jpg)

### `FIRST_VALUE()`, `LAST_VALUE()`
* Return first and last value from each window, has to be accompanied by ORDER BY
### `STDEV`